<a href="https://colab.research.google.com/github/NavdeepTU/Deep-Learning/blob/main/02.%20ML%20basics%20with%20Keras/08.%20Tune%20hyperparameters%20with%20keras%20tuner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf

In [4]:
# install and import the keras tuner
!pip install -q -U keras-tuner

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.9/128.9 kB 2.7 MB/s eta 0:00:00


In [5]:
import keras_tuner as kt

In [8]:
# download and prepare the dataset
(img_train, label_train), (img_test, label_test) = tf.keras.datasets.fashion_mnist.load_data()

In [11]:
# normalize pixel values between 0 and 1
img_train = img_train.astype('float32') / 255.0
img_test = img_test.astype('float32') / 255.0

In [1]:
# Define the model

def model_builder(hp):
  model = tf.keras.Sequential()
  model.add(tf.kears.layers.Flatten(input_shape = (28, 28)))

  # tune the number of units in the first Dense layer
  # choose an optimal value between 32 - 512
  hp_units = hp.Int('units', min_value = 32, max_value = 512, step = 32)
  model.add(tf.keras.layers.Dense(units = hp_units, activation = 'relu'))
  model.add(tf.keras.layers.Dense(10))

  # tune the learning rate for the optimizer
  # choose an optimal value from 0.01, 0.001, or 0.0001
  hp_learning_rate = hp.Choice('learning_rate', values = [1e-2, 1e-3, 1e-4])

  model.compile(
      optimizer = tf.keras.optimizers.Adam(learning_rate = hp_learning_rate),
      loss = tf.keras.losses.SparceCategoricalCrossentropy(from_logits = True),
      metrics = ['accuracy']
  )

  return model

In [ ]:
# instantiate the tuner and perform hypertuning